In [14]:
%pip install timezonefinder

Note: you may need to restart the kernel to use updated packages.


In [15]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [16]:
import os
import operator
import numpy as np
import astropy
from astropy.time import Time
from astropy.coordinates import solar_system_ephemeris, SkyCoord, EarthLocation, AltAz
from astropy.coordinates import get_body
from datetime import datetime
from timezonefinder import TimezoneFinder
import pytz

In [17]:
def get_nouns(filename):
    nouns = []
    with open(filename, 'r') as f:
        for line in f:
            nouns.append(line.strip())
    return nouns

In [18]:
def get_object_icrs(time, location, object):
    SolarSystemBodies = ['sun', 'mercury', 'venus', 'earth', 'moon', 'mars', 'jupiter', 'saturn', 'uranus', 'neptune', 'pluto']
    # Convert to lower case
    object = object.lower()
    altaz = AltAz(obstime=time, location=location)
    if object in SolarSystemBodies:
        # Get the RA and Dec of the object if in solar system
        with solar_system_ephemeris.set('builtin'): # Use for solar system bodies
            RADe_object = get_body(object, time, location) 
            # Get ICRS coordinates
            RADe_object = RADe_object.icrs
    else:
        # A celestial object in ICRS outside the solar system
        RADe_object = SkyCoord.from_name(object)
    no_interp = RADe_object.transform_to(altaz)
    az = no_interp.az.deg
    alt = no_interp.alt.deg
    print('Azimuth: ', az)
    print('Altitude: ', alt)
    # Check if viewable from location
    if alt > 0:
        print('Object is viewable from location of viewer')
    return RADe_object

In [19]:
# Get location at specific observation site
obs_location = EarthLocation.of_address('New York, NY')  #EarthLocation.of_site('greenwich') #
# EarthLocation.get_site_names()  if need to see list of sites
tz_name = TimezoneFinder().timezone_at(lng=obs_location.lon.degree,
                                       lat=obs_location.lat.degree)
tz = pytz.timezone(tz_name)
# Get time at observatory
now = datetime.now(tz)  #"2022-12-27 10:09:20.430322" #
time = Time(now) # Observation time. Convert to Astropy format

In [20]:
object = 'M3' # get_nouns(exact_object.txt) -> This needs work
download_type = 'gif' # Or 'fits'
data_source = "https://archive.stsci.edu/cgi-bin/dss_form"

In [21]:
for object in objects_list:
    RADe_object = get_object_icrs(time, obs_location, object) # Or 'sun' if looking in solar system but sky survey cant retrieve solar system image
    RA = RADe_object.ra.deg
    DE = RADe_object.dec.deg
    print('RA = ', RA)
    print('DE = ', DE)

Azimuth:  130.86740301515917
Altitude:  72.54461707508963
Object is viewable from location of viewer
RA =  205.548417
DE =  28.377278
